<a href="https://colab.research.google.com/github/manal497/data_cleaning/blob/main/vintage_cleaning_db.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
fp = 'https://bit.ly/vintage-auctions'

In [ ]:
fp

'https://bit.ly/vintage-auctions'

In [ ]:
df = pd.read_csv(fp)

In [ ]:
df.dtypes

tx id         object
time          object
auction       object
color         object
shirt size    object
condition     object
dtype: object

In [ ]:
df.head()

,tx id,time,auction,color,shirt size,condition
0,8dbf3cfa48b25f32e300f279bb4163c67d5c60cd,3:36 PM,$17.18,Puce,S,used
1,8ccebfdb1a41830bdcc0e926531f3215986b3a90,9:03 AM,$15.19,Aquamarine,L,never worn
2,ae85a421a91cec8baf16ff1e45fb3e239130b584,7:42 PM,$7.58,Violet,L,used
3,85e44f09a04a79182befcfd55e288a855f633e81,10:28 PM,$9.10,Goldenrod,XS,NaN
4,ff750bfca4910cfa7f538fbe5463457839021680,6:38 AM,$8.22,NaN,2XL,used


In [ ]:
df.shape

(1000, 6)

In [ ]:

df.isna().sum()

tx id           0
time            0
auction         0
color          23
shirt size      0
condition     153
dtype: int64

In [ ]:
df['time'] = pd.to_datetime(df['time'])

In [ ]:
df['time']

0     2022-01-23 15:36:00
1     2022-01-23 09:03:00
2     2022-01-23 19:42:00
3     2022-01-23 22:28:00
4     2022-01-23 06:38:00
              ...        
995   2022-01-23 09:56:00
996   2022-01-23 11:31:00
997   2022-01-23 06:25:00
998   2022-01-23 02:22:00
999   2022-01-23 19:47:00
Name: time, Length: 1000, dtype: datetime64[ns]

In [ ]:
df['auction'] = df['auction'].str.replace('$','').astype(float)

In [ ]:
df['color'] = df['color'].astype('category')

In [ ]:
df['shirt size'] = df['shirt size'].astype('category')

In [ ]:
df['condition'] = df['condition'].astype('category')

In [ ]:
df.duplicated().sum()

0

In [ ]:
df['time'].dt.hour

0      15
1       9
2      19
3      22
4       6
       ..
995     9
996    11
997     6
998     2
999    19
Name: time, Length: 1000, dtype: int64

In [ ]:
df['time'].dt.minute

0      36
1       3
2      42
3      28
4      38
       ..
995    56
996    31
997    25
998    22
999    47
Name: time, Length: 1000, dtype: int64

nuit 1h 6h29
matin 6h29 11h59
midi 12h 13h59
apres midi 14h 17h59
after work 18h 20h59
soir 21h 0h59

In [ ]:
tmp = df['time'].dt.hour * 100 + df['time'].dt.minute

In [ ]:
tmp = pd.cut(tmp,
       bins=[0, 59, 629, 1159, 1359, 1759, 2059, 2400],
       labels=['soir', 'nuit', 'matin', 'midi', 'apres midi', 'after work', 'soir2'])

In [ ]:
#enlever le 2 du soir 2
df['time_cat'] = tmp.str.replace('2', '').astype('category')

In [ ]:
#drop inulile
df.drop(['time', 'tx id'], axis='columns', inplace=True)

Encoder tout les variable categorique en OneHot en indiquant l'absance des valeurs

In [ ]:
df = pd.get_dummies(df, dummy_na=True)

In [ ]:

df.dtypes

auction                 float64
color_Aquamarine          uint8
color_Blue                uint8
color_Crimson             uint8
color_Fuscia              uint8
color_Goldenrod           uint8
color_Green               uint8
color_Indigo              uint8
color_Khaki               uint8
color_Maroon              uint8
color_Mauv                uint8
color_Orange              uint8
color_Pink                uint8
color_Puce                uint8
color_Purple              uint8
color_Red                 uint8
color_Teal                uint8
color_Turquoise           uint8
color_Violet              uint8
color_Yellow              uint8
color_nan                 uint8
shirt size_2XL            uint8
shirt size_3XL            uint8
shirt size_L              uint8
shirt size_M              uint8
shirt size_S              uint8
shirt size_XL             uint8
shirt size_XS             uint8
shirt size_nan            uint8
condition_Never Worn      uint8
condition_Used            uint8
conditio

In [ ]:
#time cat et shirt size n'ont pas val nan
df.drop(['time_cat_nan', 'shirt size_nan'], axis='columns', inplace=True)

##split train/test/valid par rapport a la colone time cat 

In [ ]:
df.columns[df.columns.str.contains('time_cat')]

Index(['time_cat_after work', 'time_cat_apres midi', 'time_cat_matin',
       'time_cat_midi', 'time_cat_nuit', 'time_cat_soir'],
      dtype='object')

In [ ]:
time_cats = df.columns[df.columns.str.contains('time_cat')]

In [ ]:
df.head(220)

,auction,color_Aquamarine,color_Blue,color_Crimson,color_Fuscia,color_Goldenrod,color_Green,color_Indigo,color_Khaki,color_Maroon,color_Mauv,color_Orange,color_Pink,color_Puce,color_Purple,color_Red,color_Teal,color_Turquoise,color_Violet,color_Yellow,color_nan,shirt size_2XL,shirt size_3XL,shirt size_L,shirt size_M,shirt size_S,shirt size_XL,shirt size_XS,condition_Never Worn,condition_Used,condition_never worn,condition_used,condition_nan,time_cat_after work,time_cat_apres midi,time_cat_matin,time_cat_midi,time_cat_nuit,time_cat_soir
0,17.18,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0
1,15.19,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0
2,7.58,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
3,9.10,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1
4,8.22,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,16.45,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0
216,2.90,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0
217,9.03,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0
218,6.63,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0


In [ ]:
#train
train_parts= []
for col_name in time_cats:
  part = df[df[col_name]==1].sample(frac=0.9, random_state=100)
  train_parts.append(part)

train_set = pd.concat(train_parts, axis='index')  

In [ ]:
#trouver les ligne qui ne sont pas dans le train
not_in_train = ~df.index.isin(train_set.index)

In [ ]:

test_set = df[not_in_train]

In [ ]:
#valid
valid_parts= []
for col_name in time_cats:
  part = train_set[train_set[col_name]==1].sample(frac=0.2, random_state=101)
  valid_parts.append(part)

valid_set = pd.concat(valid_parts, axis='index')  

In [ ]:
valid_set.shape

(180, 39)

In [ ]:
#receer notre train set 
#just les val qui sont dans train set mais pas dans valid set~
not_in_valid = ~train_set.index.isin(valid_set.index)


train_set = train_set[not_in_valid]



In [ ]:
train_set

,auction,color_Aquamarine,color_Blue,color_Crimson,color_Fuscia,color_Goldenrod,color_Green,color_Indigo,color_Khaki,color_Maroon,color_Mauv,color_Orange,color_Pink,color_Puce,color_Purple,color_Red,color_Teal,color_Turquoise,color_Violet,color_Yellow,color_nan,shirt size_2XL,shirt size_3XL,shirt size_L,shirt size_M,shirt size_S,shirt size_XL,shirt size_XS,condition_Never Worn,condition_Used,condition_never worn,condition_used,condition_nan,time_cat_after work,time_cat_apres midi,time_cat_matin,time_cat_midi,time_cat_nuit,time_cat_soir
651,9.41,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0
282,10.49,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0
383,9.87,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
878,6.76,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
193,16.85,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
459,16.90,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1
485,16.60,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
766,3.97,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1
823,18.98,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1


##isoler le variable cible auction

In [ ]:
Y_train = train_set['auction']

In [ ]:
X_train = train_set.drop(['auction'], axis='columns')